In [67]:
import os
import pickle
import torch
import numpy as np
from math import ceil
from model_vc import Generator
import librosa
import librosa.display
import matplotlib.pyplot as plt

import math
import glob

baseline = pickle.load(open('/home/ytang363/7100_voiceConversion/002_baseline-results.pkl', "rb"))
retrain = pickle.load(open('/home/ytang363/7100_voiceConversion/002_results.pkl', "rb"))

In [68]:
# function: https://dsp.stackexchange.com/questions/56391/mel-cepstral-distortion
def log_spec_dB_dist(x, y):
    log_spec_dB_const = 10.0 / math.log(10.0) * math.sqrt(2.0)
    diff = x - y
    return log_spec_dB_const * math.sqrt(np.inner(diff, diff))

cost_function = log_spec_dB_dist

In [69]:
# Load ground truth utterance
rootDir = '/home/ytang363/7100_voiceConversion/VCTK-Corpus-0.92/spmel-16k'
dirName, subdirList, _ = next(os.walk(rootDir))
process_speakers = ['p225', 'p226', 'p227', 'p228', 'p229', 'p230', 'p231', 'p232', 'p233', 'p234', 'p237']
process_uttr = '001'
speakers = []
gt = {}

for speaker in sorted(subdirList):
    if speaker not in process_speakers:
        continue

    print('Processing speaker: %s' % speaker)
    _, _, fileList = next(os.walk(os.path.join(dirName,speaker)))

    indices = [i for i, element in enumerate(fileList) if process_uttr in element]
    print(fileList[indices[0]]) 

    spec = np.load(os.path.join(dirName, speaker, fileList[indices[0]]))

    gt[speaker] = spec

Processing speaker: p225
p225_001_mic1.npy
Processing speaker: p226
p226_001_mic1.npy
Processing speaker: p227
p227_001_mic1.npy
Processing speaker: p228
p228_001_mic1.npy
Processing speaker: p229
p229_001_mic1.npy
Processing speaker: p230
p230_001_mic1.npy
Processing speaker: p231
p231_001_mic1.npy
Processing speaker: p232
p232_001_mic1.npy
Processing speaker: p233
p233_001_mic1.npy
Processing speaker: p234
p234_001_mic1.npy
Processing speaker: p237
p237_001_mic1.npy


In [70]:
# Loop for evaluation
def avg_mcd(gen_mel):
    total_diff = 0
    for i in range(len(process_speakers)):
        current = gen_mel[i][0].split('x')[1]
        test_uttr = gen_mel[i][1] # for p225-001, the shape is (129,80)
        gt_uttr = gt[current]

        min_cost, _ = librosa.sequence.dtw(gt_uttr[:, 1:].T, test_uttr[:, 1:].T, 
                                            metric=cost_function)  
        print(gt_uttr.shape)
        print(test_uttr.shape)
        print(gt_uttr[:, 1:].shape)
        print('----')                                                                            
        diff = np.mean(min_cost) / gt_uttr.shape[0]
        total_diff += diff
    avg_MCD = total_diff / len(process_speakers)
    return avg_MCD       

In [71]:
avg_mcd(baseline)

(129, 80)
(247, 80)
(129, 79)
----
(175, 80)
(247, 80)
(175, 79)
----
(215, 80)
(247, 80)
(215, 79)
----
(167, 80)
(247, 80)
(167, 79)
----
(131, 80)
(247, 80)
(131, 79)
----
(139, 80)
(247, 80)
(139, 79)
----
(128, 80)
(247, 80)
(128, 79)
----
(132, 80)
(247, 80)
(132, 79)
----
(164, 80)
(247, 80)
(164, 79)
----
(174, 80)
(247, 80)
(174, 79)
----
(144, 80)
(247, 80)
(144, 79)
----


7.379201538714358

In [54]:
avg_mcd(retrain)

4.815679425487986